In [1]:
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "OECD_file/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = pd.read_csv(file_name, delimiter="|") # load the data
        print(df)

          appln_id        app_nbr  filing  tech_field  many_field  \
0         16427085  EP19780100001    1978        16.0           1   
1         16427086  EP19780100002    1978        31.0           0   
2         16427087  EP19780100003    1978        32.0           1   
3         16427088  EP19780100004    1978        14.0           1   
4         16427089  EP19780100005    1978         8.0           0   
...            ...            ...     ...         ...         ...   
3507894  506385435  EP20190405001    2019         8.0           1   
3507895  506385437  EP20190405002    2019        24.0           0   
3507896  507537560  EP20190460001    2019        35.0           0   
3507897  505638106  EP20190461501    2019         1.0           1   
3507898  506424754  EP20190700348    2019         6.0           0   

         patent_scope  family_size  grant_lag  bwd_cits  npl_cits  ...  \
0                   3            7     1203.0         4         0  ...   
1                   1  

In [2]:
# Now try to pull the patent data combining 2 files to accumulate 10 years
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "Patent_EP/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

patent_data_combine = None

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        patent_data = pd.read_csv(file_name) # load the data
        # print(patent_data)
        if patent_data_combine is None:
            patent_data_combine = patent_data
        else:
            patent_data_combine = patent_data_combine.append(patent_data)

In [3]:
patent_data_combine.sort_values('filing_date')

,publication_number,application_number,text,filing_date
161697,EP-2393505-A1,EP-10705790-A,The present invention relates to the use of a ...,20100102
11332,EP-2376972-A1,EP-10708886-A,"A parallel-sighted, image-reversing prism syst...",20100102
52710,EP-2384189-A2,EP-10726819-A,Improved compositions for tissue augmentation ...,20100103
74016,EP-2374013-A2,EP-10708389-A,Apparatus for radiation mapping of a region co...,20100103
157739,EP-2374128-A2,EP-10703964-A,A device for recording at least part of a data...,20100103
...,...,...,...,...
16172,EP-3572775-A2,EP-19193474-A,A switch control circuit and method for an ele...,20190823
19388,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
19389,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
12887,EP-3564854-A1,EP-19196510-A,Embodiments of the present invention provide a...,20190910


In [4]:
patent_abstract = pd.read_csv('patent_abstract_5000_2015.csv')
patent_abstract.shape

(5000, 32)

In [10]:
#
#conditions = [
#    (df_merge_quality['quality_index_6'] >= 0) & (df_merge_quality['quality_index_6'] < 0.15),
#    (df_merge_quality['quality_index_6'] >= 0.15) & (df_merge_quality['quality_index_6'] < 0.3),
#    (df_merge_quality['quality_index_6'] >= 0.3) & (df_merge_quality['quality_index_6'] < 0.45),
#    (df_merge_quality['quality_index_6'] >= 0.45) & (df_merge_quality['quality_index_6'] < 0.6),
#    (df_merge_quality['quality_index_6'] >= 0.6)
#]

#choices = ['0', '1', '2', '3', '4']
#**//

In [5]:
conditions = [

    (patent_abstract['quality_index_4'] < 0.3),
    (patent_abstract['quality_index_4'] >= 0.3)
]

choices = ['0', '1']

In [6]:
import numpy as np
patent_abstract['quality_rank'] = np.select(condlist=conditions, choicelist=choices)

In [7]:
patent_abstract['quality_rank'].value_counts()

0    3283
1    1717
Name: quality_rank, dtype: int64

In [12]:
data_train = patent_abstract

In [8]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [10]:
MAX_SENT_LENGTH = 30
MAX_SENTS = 3
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [11]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [13]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence 
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data)
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [14]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [15]:
tokenizer.word_counts

OrderedDict([('provided', 1622),
             ('herein', 133),
             ('are', 2841),
             ('methods', 441),
             ('of', 22194),
             ('using', 641),
             ('gaseous', 20),
             ('nitric', 8),
             ('oxide', 135),
             ('and', 18375),
             ('or', 4479),
             ('at', 4946),
             ('least', 3320),
             ('one', 3870),
             ('source', 448),
             ('to', 15922),
             ('decrease', 25),
             ('sebum', 4),
             ('production', 143),
             ('in', 9848),
             ('the', 51109),
             ('skin', 80),
             ('a', 37245),
             ('subject', 138),
             ('some', 122),
             ('embodiments', 231),
             ('includes', 1719),
             ('small', 66),
             ('molecule', 29),
             ('macromolecular', 5),
             ('no', 132),
             ('releasing', 31),
             ('compounds', 228),
             ('discl

In [16]:
tokenizer.document_count

5000

In [17]:
len(tokenizer.word_index) # 143937 unqiue vocabulary out of the entire abstract dataset

17545

In [18]:
for i, sentences in enumerate(abstracts):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
            #for word in wordTokens:
                if word in tokenizer.word_index.keys():
                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

In [19]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 17545 unique tokens.


In [20]:
np.unique(np.asarray(labels))

array(['0', '1'], dtype='<U1')

In [21]:
labels = to_categorical(np.asarray(labels), num_classes=2) 
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 3, 30)
Shape of label tensor: (5000, 2)


In [22]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [23]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[2623. 1377.]
[660. 340.]


In [24]:
x_train[2]

array([[   1,  228,  127,   49,   46,    2,  617,   41,    9,  126,    2,
        1056,  123,    4,    2,   86,   41,   45,   33,    2,  123,  383,
         325,   15,    6,   90,    7,    1,  134,    3],
       [   1,  123,  134,  241,    6,  445,   16,    1,  123,  383,    4,
           1,  123,  134,  776,    6,  445,   83,    1,  414,  420,  383,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [25]:
x_val[10]

array([[    2,   217,    33,     2,    72,     3,  6756,    35,    10,
           20,    14,   978,    46,     2,   554,  2700,   158,     4,
           10,    20,    14,   978,    46,     2,  2700, 17346, 17347,
           15,     6,     7],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]], dtype=int32)

In [26]:
x_train.shape

(4000, 3, 30)

In [37]:
np.unique(y_val, return_counts=True)

(array([0., 1.], dtype=float32), array([1000, 1000]))

In [27]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops

import scipy.sparse


# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import glove_helper; reload(glove_helper)

<module 'glove_helper' from '/home/nobu_yamaguchi/final_project/glove_helper.py'>

In [28]:
GLOVE_DIR = "data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [29]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [76]:

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

(17546, 100)


In [77]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_sent)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_dense)
l_lstm_sent_drop = Dropout(rate=0.5)(l_lstm_dense)
preds = Dense(2, activation='sigmoid')(l_lstm_sent_drop) # [Steven] why 5 will throw error and require 6 ? it is weird since we only have 5 number scale
model = Model(review_input, preds)

In [78]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [79]:
print("model fitting - Hierachical LSTM")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=5, batch_size=50)

model fitting - Hierachical LSTM
Train on 4000 samples, validate on 1000 samples
Epoch 1/5
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6527 - acc: 0.6543 - val_loss: 0.6436 - val_acc: 0.6600
Epoch 2/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.6473 - acc: 0.6557 - val_loss: 0.6415 - val_acc: 0.6600
Epoch 3/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.6457 - acc: 0.6556 - val_loss: 0.6409 - val_acc: 0.6600
Epoch 4/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.6394 - acc: 0.6554 - val_loss: 0.6413 - val_acc: 0.6600
Epoch 5/5
4000/4000 [==============================] - 9s 2ms/step - loss: 0.6350 - acc: 0.6550 - val_loss: 0.6439 - val_acc: 0.6600
